<a href="https://colab.research.google.com/github/Saku052/YoutubeindustryReserch/blob/main/Youtube_SDIndustry_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from googleapiclient.discovery import build
from IPython.display import JSON
import pandas as pd

In [58]:
api_key = ''
channel_ids = ['UCtLc0LqPp8aq4_cvjBDVfCw', #ぺんたろう
              'UCtw4S54qOeBW1inR5--jSFQ', #ぷりずんだAI
              'UCAajXFu19cCUvSJW2FdHcwg', #しろぼうVtuber
              'UCYqTtuRm0H3roAL0iJd1CaA', #キシリのお絵かきチャンネル
              'UCmqz9u0Fp099PZr3Bb_GZ9A', #動き続けるCAN
              'UClz-2IhWHgFNw7gnHe6Xgyw', #だだっこぱんだ
]
youtube = build('youtube', 'v3', developerKey = api_key)

## チャンネルの取得

In [69]:
def get_channel_stats(youtube, channel_id):
  all_data = []
  
  request = youtube.channels().list(
      part="snippet,contentDetails,statistics",
        id= ','.join(channel_id)
  )
  response = request.execute()

  for i in range(len(response['items'])):
    data = dict(Channel_name = response['items'][i]['snippet']['title'],
                Subscribers = response['items'][i]['statistics']['subscriberCount'],
                Views = response['items'][i]['statistics']['viewCount'],
                Total_videos = response['items'][i]['statistics']['videoCount'],
                playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
    all_data.append(data)

  return all_data

In [71]:
CH_data = pd.DataFrame(get_channel_stats(youtube, channel_ids))
CH_data

,Channel_name,Subscribers,Views,Total_videos,playlist_id
0,ペンたろうのAIイラストお絵描きチャンネル,551,43330,20,UUtLc0LqPp8aq4_cvjBDVfCw
1,ぷりずんだAI,3550,52911,9,UUtw4S54qOeBW1inR5--jSFQ
2,キシリのお絵描きチャンネル,1580,146265,148,UUYqTtuRm0H3roAL0iJd1CaA
3,しろぼう【Vtuber講座】,4140,406294,28,UUAajXFu19cCUvSJW2FdHcwg
4,動き続けるCAN,392,47853,26,UUmqz9u0Fp099PZr3Bb_GZ9A
5,だだっこぱんだ,2270,632235,91,UUlz-2IhWHgFNw7gnHe6Xgyw


## 動画リストの取得

In [72]:
playlist_id = CH_data.loc[CH_data['Channel_name'] == 'ぷりずんだAI', 'playlist_id'].iloc[0]
playlist_id

'UUtw4S54qOeBW1inR5--jSFQ'

In [86]:
def get_video_ids(youtube, playlist_id):
  video_ids = []

  request = youtube.playlistItems().list(
      part = 'contentDetails',
      playlistId = playlist_id,
      maxResults = 30)
  response = request.execute()

  for i in range(len(response['items'])):
    video_ids.append(response['items'][i]['contentDetails']['videoId'])

  return video_ids

In [92]:
video_ids = get_video_ids(youtube, playlist_id)

## 動画メタデータの取得

In [104]:
def get_video_details(youtube, video_ids):
  all_video_stats = []

  request = youtube.videos().list(
      part='snippet,statistics',
      id=','.join(video_ids)
  )
  response = request.execute()

  for video in response['items']:
    video_stats = dict(Title = video['snippet']['title'],
                       published_date = video['snippet']['publishedAt'],
                       Views = video['statistics']['viewCount'],
                       Likes = video['statistics']['likeCount'],
                       Comments = video['statistics']['commentCount'])
    all_video_stats.append(video_stats)

  return all_video_stats

In [107]:
video_data = pd.DataFrame(get_video_details(youtube, video_ids))
video_data

,Title,published_date,Views,Likes,Comments
0,【VOICEVOX解説】ずんだもんと学ぶ「AI を受け入れたサイトで起きている問題」【Nov...,2022-12-03T06:39:47Z,3901,100,52
1,Novel AIを１０倍楽しく、楽にするお役立ちツールを5選紹介！新しい魔導書も解説！【ずん...,2022-11-17T10:37:38Z,2957,141,6
2,アニメ音声合成AI「MoeGoe」出現！声優のお仕事も失われる!? 使い方解説!!【ずんだも...,2022-11-08T14:30:39Z,2059,58,19
3,動画もAIによってヤバくなる!? 今開発中の動画AIを紹介!!【ずんだもんAI解説】,2022-11-03T09:46:26Z,1877,68,13
4,Novel AIを越えろ！ 次に来るAIをずんだもんが解説！【ずんだもんAI解説】,2022-10-29T06:18:25Z,4036,140,23
5,【VOICEVOX解説】ずんだもんと学ぶ「イラストAI でどんな金稼ぎが発生しているのか!?...,2022-10-24T12:48:57Z,5499,124,31
6,【VOICEVOX解説】ずんだもんと学ぶ「『元素法典』でNovelAIを進化させる!! 使い...,2022-10-18T13:06:04Z,5392,217,13
7,【VOICEVOX解説】ずんだもんと学ぶ「Novel AIでクオリティの高いイラストを出すテ...,2022-10-16T07:05:37Z,9999,391,39
8,【VOICEVOX解説】ずんだもんと学ぶイラストAIの歴史と問題点【NovelAI】,2022-10-12T11:12:43Z,17229,324,93
